<a href="https://colab.research.google.com/github/gglilord8-cmyk/Harry/blob/main/avatarify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Avatarify Colab Server

This Colab notebook is for running Avatarify rendering server. It allows you to run Avatarify on your computer **without GPU** in this way:

1. When this notebook is executed, it starts listening for incoming requests from your computer;
1. You start the client on your computer and it connects to the notebook and starts sending requests;
1. This notebooks receives the requests from your computer, renders avatar images and sends them back;

To this end, all the heavy work is offloaded from your computer to this notebook so you don't need to have a beafy hardware on your PC anymore.


## Start the server
Run the cells below (Shift+Enter) sequentially and pay attention to the hints and instructions included in this notebook.

At the end you will get a command for running the client on your computer.

## Start the client

Make sure you have installed the latest version of Avatarify on your computer. Refer to the [README](https://github.com/alievk/avatarify#install) for the instructions.

When it's ready execute this notebook and get the command for running the client on your computer.


### Technical details

The client on your computer connects to the server via `ngrok` TCP tunnel or a reverse `ssh` tunnel.

`ngrok`, while easy to use, can induce a considerable network lag ranging from dozens of milliseconds to a second. This can lead to a poor experience.

A more stable connection could be established using a reverse `ssh` tunnel to a host with a public IP, like an AWS `t3.micro` (free) instance. This notebook provides a script for creating a tunnel, but launching an instance in a cloud is on your own (find the manual below).

# Install

### Avatarify
Follow the steps below to clone Avatarify and install the dependencies.

In [114]:
!cd /content
!rm -rf *

In [116]:
!git clone https://github.com/alievk/avatarify.git

Cloning into 'avatarify'...
remote: Enumerating objects: 1514, done.
remote: Total 1514 (delta 0), reused 0 (delta 0), pack-reused 1514 (from 1)
Receiving objects: 100% (1514/1514), 5.69 MiB | 26.47 MiB/s, done.
Resolving deltas: 100% (967/967), done.


In [119]:
cd avatarify

[Errno 2] No such file or directory: 'avatarify'
/content/avatarify/avatarify


In [163]:
!git clone https://github.com/alievk/first-order-model.git fomm
!pip install face-alignment==1.0.0 msgpack_numpy

fatal: destination path 'fomm' already exists and is not an empty directory.


In [164]:
!scripts/download_data.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  228M  100  228M    0     0   131M      0  0:00:01  0:00:01 --:--:--  131M
Expected checksum: 8a45a24037871c045fbb8a6a8aa95ebc
Found checksum:    8a45a24037871c045fbb8a6a8aa95ebc  vox-adv-cpk.pth.tar


### ngrok
Follow the steps below to setup ngrok. You will also need to sign up on the ngrok site and get your authtoken (free).


In [135]:
# Download ngrok
!/content/avatarify/avatarify/scripts/get_ngrok.sh
!ls -F /content/avatarify/avatarify/scripts
!ls -l /content/avatarify/avatarify/ngrok # Check ngrok binary presence and permissions

ngrok is not found, installing...
Done!
create_virtual_camera.sh*  install_mac.sh*	  open_tunnel_ssh.sh*
download_data.sh*	   install.sh*		  settings.sh*
get_ngrok.sh*		   install_windows.bat	  settings_windows.bat
install_docker.sh	   open_tunnel_ngrok.sh*
-rwxr-xr-x 1 root root 30888120 Dec 11 20:27 /content/avatarify/avatarify/ngrok


In [139]:
%%writefile /content/avatarify/avatarify/scripts/open_tunnel_ngrok.sh
#!/bin/bash
set -ex # Enable debugging and exit on error

echo "[DEBUG] Starting open_tunnel_ngrok.sh"

# Kill previous ngrok processes
echo "[DEBUG] Attempting to kill previous ngrok processes..."
killall ngrok || true # Continue even if no processes to kill
echo "[DEBUG] Finished killing ngrok processes."

# Start ngrok, redirecting output to a log file
echo "[DEBUG] Current directory for ngrok start: $(pwd)"
echo "[DEBUG] Starting ngrok with config ngrok.conf"
./ngrok start --all --config ngrok.conf > ngrok_output.log 2>&1 &
echo "[DEBUG] Ngrok process started in background."

echo "[DEBUG] open_tunnel_ngrok.sh finished."


Overwriting /content/avatarify/avatarify/scripts/open_tunnel_ngrok.sh


# Run
Start here if the runtime was restarted after installation.

In [123]:
cd /content/avatarify

/content/avatarify


In [124]:
#!git pull origin

In [125]:
from subprocess import Popen, PIPE
import shlex
import json
import time

def run_with_pipe(command):
  commands = list(map(shlex.split,command.split("|")))
  ps = Popen(commands[0], stdout=PIPE, stderr=PIPE)
  for command in commands[1:]:
    ps = Popen(command, stdin=ps.stdout, stdout=PIPE, stderr=PIPE)
  return ps.stdout.readlines()


def get_tunnel_adresses():
  print("Attempting to get tunnel addresses...")
  raw_info = run_with_pipe("curl http://localhost:4040/api/tunnels")
  print(f"Raw curl output: {raw_info}")
  assert raw_info, "Curl command returned no info."

  try:
    info = json.loads(raw_info[0])
    print(f"Parsed JSON info: {info}")
  except json.JSONDecodeError as e:
    print(f"JSON decoding error: {e}")
    raise ValueError("Failed to decode JSON from ngrok API.") from e

  in_addr = None
  out_addr = None

  if 'tunnels' not in info:
      raise ValueError("No 'tunnels' key found in ngrok API response.")

  for tunnel in info['tunnels']:
    url = tunnel['public_url']
    port = url.split(':')[-1]
    local_port = tunnel['config']['addr'].split(':')[-1]
    print(f'{url} -> {local_port} [{tunnel["name"]}]')
    if tunnel['name'] == 'input':
      in_addr = url
    elif tunnel['name'] == 'output':
      out_addr = url
    else:
      print(f'unknown tunnel: {tunnel["name"]}')

  if in_addr is None or out_addr is None:
      raise ValueError("Input or Output tunnel not found in ngrok API response.")

  return in_addr, out_addr

In [19]:
# Input and output ports for communication
local_in_port = 5557
local_out_port = 5558

# Start the worker


In [158]:
import os
from subprocess import Popen, PIPE
import shlex
import time

# Ensure we are in the correct directory
os.chdir('/content/avatarify/avatarify')

# Ensure run.sh exists and is executable
if not os.path.exists('./run.sh'):
    print("Error: run.sh not found. Please ensure Avatarify was cloned correctly.")
else:
    os.chmod('./run.sh', 0o755) # Make it executable
    print("run.sh found and set to executable.")

# (Re)Start the worker
with open('/tmp/run.txt', 'w') as f:
  ps = Popen(
      shlex.split(f'./run.sh --is-worker --in-port {local_in_port} --out-port {local_out_port} --no-vcam --no-conda'),
      stdout=f, stderr=f)
  time.sleep(3)

run.sh found and set to executable.


In [153]:
!ls -F /content/avatarify

avatarify/  ngrok.conf


This command should print lines if the worker is successfully started

In [154]:
!ps aux | grep 'python3 afy/cam_fomm.py' | grep -v grep | tee /tmp/ps_run
!if [[ $(cat /tmp/ps_run | wc -l) == "0" ]]; then echo "Worker failed to start"; cat /tmp/run.txt; else echo "Worker started"; fi

Worker failed to start


In [156]:
!ls -F /content/avatarify/avatarify

afy/		 LICENSE.md			  requirements.txt
avatarify.ipynb  ngrok*				  run_mac.sh*
avatars/	 ngrok.conf			  run.sh*
config.yaml	 ngrok_output.log		  run_windows.bat
Dockerfile	 ngrok-v3-stable-linux-amd64.tgz  scripts/
docs/		 README.md			  var/
fomm/		 requirements_client.txt	  vox-adv-cpk.pth.tar


# Open ngrok tunnel

#### Get ngrok token
Go to https://dashboard.ngrok.com/auth/your-authtoken (sign up if required), copy your authtoken and put it below.

In [150]:
# Paste your authtoken here in quotes
authtoken = "YOUR_ACTUAL_NGROK_AUTHTOKEN_HERE"

Set your region

Code | Region
--- | ---
us | United States
eu | Europe
ap | Asia/Pacific
au | Australia
sa | South America
jp | Japan
in | India

In [137]:
# Set your region here in quotes
region = "eu"

In [138]:
import os

config =\
f"""
version: 2
authtoken: {authtoken}
region: {region}
console_ui: False
tunnels:
  input:
    addr: {local_in_port}
    proto: tcp
  output:
    addr: {local_out_port}
    proto: tcp
"""

# Ensure ngrok.conf is written to the expected location by ngrok
config_path = os.path.join('/content/avatarify/avatarify', 'ngrok.conf')
with open(config_path, 'w') as f:
  f.write(config)

In [157]:
import os
import time
from subprocess import Popen, PIPE, TimeoutExpired

# Ensure we are in the correct directory for ngrok scripts
os.chdir('/content/avatarify/avatarify')

print(f"Current working directory: {os.getcwd()}")

# Construct absolute path to the script
script_path = os.path.join(os.getcwd(), 'scripts', 'open_tunnel_ngrok.sh')
print(f"Attempting to run script from: {script_path}")

# (Re)Open tunnel
proc = Popen(f'/bin/bash "{script_path}"', shell=True, stdout=PIPE, stderr=PIPE)
try:
  # Use communicate to wait for the bash script to exit, and capture its output
  stdout_val, stderr_val = proc.communicate(timeout=60)
  print(f"\n--- open_tunnel_ngrok.sh stdout ---\n{stdout_val.decode()}")
  print(f"\n--- open_tunnel_ngrok.sh stderr ---\n{stderr_val.decode()}")
  print(f"open_tunnel_ngrok.sh exit code: {proc.returncode}")
except TimeoutExpired:
  proc.kill()
  stdout_val, stderr_val = proc.communicate()
  print(f"\n--- open_tunnel_ngrok.sh stdout (Timeout) ---\n{stdout_val.decode()}")
  print(f"\n--- open_tunnel_ngrok.sh stderr (Timeout) ---\n{stderr_val.decode()}")
  print("Error: open_tunnel_ngrok.sh timed out.")


time.sleep(45) # Give ngrok time to fully initialize and write to its log

# Read ngrok output log
print("\n--- ngrok_output.log content ---\n")
if os.path.exists('ngrok_output.log'):
    with open('ngrok_output.log', 'r') as f:
        print(f.read())
else:
    print("ngrok_output.log not found.")

# Check if ngrok process is running
print("\nChecking for ngrok process:\n")
!ps aux | grep ngrok | grep -v grep

Current working directory: /content/avatarify/avatarify
Attempting to run script from: /content/avatarify/avatarify/scripts/open_tunnel_ngrok.sh

--- open_tunnel_ngrok.sh stdout ---
[DEBUG] Starting open_tunnel_ngrok.sh
[DEBUG] Attempting to kill previous ngrok processes...
[DEBUG] Finished killing ngrok processes.
[DEBUG] Current directory for ngrok start: /content/avatarify/avatarify
[DEBUG] Starting ngrok with config ngrok.conf
[DEBUG] Ngrok process started in background.
[DEBUG] open_tunnel_ngrok.sh finished.


--- open_tunnel_ngrok.sh stderr ---
+ echo '[DEBUG] Starting open_tunnel_ngrok.sh'
+ echo '[DEBUG] Attempting to kill previous ngrok processes...'
+ killall ngrok
ngrok: no process found
+ true
+ echo '[DEBUG] Finished killing ngrok processes.'
++ pwd
+ echo '[DEBUG] Current directory for ngrok start: /content/avatarify/avatarify'
+ echo '[DEBUG] Starting ngrok with config ngrok.conf'
+ echo '[DEBUG] Ngrok process started in background.'
+ echo '[DEBUG] open_tunnel_ngrok.sh 

In [162]:
!pip install pyyaml

In [159]:
!ps aux | grep 'python3 afy/cam_fomm.py' | grep -v grep | tee /tmp/ps_run
!if [[ $(cat /tmp/ps_run | wc -l) == "0" ]]; then echo "Worker failed to start"; cat /tmp/run.txt; else echo "Worker started"; fi

Worker failed to start
[1767043000.790471] Loading Predictor
/content/avatarify/avatarify/fomm/modules/dense_motion.py:71: SyntaxWarning: invalid escape sequence '\h'
  Eq 7. in the paper \hat{T}_{s<-d}(z)
Traceback (most recent call last):
  File "/content/avatarify/avatarify/afy/cam_fomm.py", line 201, in <module>
    from afy import predictor_worker
  File "/content/avatarify/avatarify/afy/predictor_worker.py", line 1, in <module>
    from predictor_local import PredictorLocal
  File "/content/avatarify/avatarify/afy/predictor_local.py", line 8, in <module>
    import face_alignment
ModuleNotFoundError: No module named 'face_alignment'


In [142]:
# Get tunnel addresses
try:
  in_addr, out_addr = get_tunnel_adresses()
  print("Tunnel opened")
except Exception as e:
  [print(l.decode(), end='') for l in ps.stdout.readlines()]
  print("Something went wrong, reopen the tunnel")
  print(f"Error: {type(e).__name__}: {e}")

Attempting to get tunnel addresses...
Raw curl output: []
Something went wrong, reopen the tunnel
Error: AssertionError: Curl command returned no info.


### [Optional] AWS proxy
Alternatively you can create a ssh reverse tunnel to an AWS `t3.micro` instance (it's free). It has lower latency than ngrok.

1. In your AWS console go to Services -> EC2 -> Instances -> Launch Instance;
1. Choose `Ubuntu Server 18.04 LTS` AMI;
1. Choose `t3.micro` instance type and press Review and launch;
1. Confirm your key pair and press Launch instances;
1. Go to the security group of this instance and edit inbound rules. Add TCP ports 5557 and 5558 and set Source to Anywhere. Press Save rules;
1. ssh into the instance (you can find the command in the Instances if you click on the Connect button) and add this line in the end of `/etc/ssh/sshd_config`:
```
GatewayPorts yes
```
then restart `sshd`
```
sudo service sshd restart
```
1. Copy your `key_pair.pem` by dragging and dropping it into avatarify folder in this notebook;
1. Use the command below to open the tunnel;
1. Start client with a command (substitute `run_mac.sh` with `run_windows.bat` or `run.sh`)
```
./run_mac.sh --is-client --in-addr tcp://instace.compute.amazonaws.com:5557 --out-addr tcp://instance.compute.amazonaws.com:5558
```

In [55]:
# Open reverse ssh tunnel (uncomment line below)
# !./scripts/open_tunnel_ssh.sh key_pair.pem ubuntu@instance.compute.amazonaws.com

# Start the client
When you run the cell below it will print a command. Run this command on your computer:

1. Open a terminal (in Windows open `Anaconda Prompt`);
2. Change working directory to the `avatarify` directory:</br>
* Windows (change `C:\path\to\avatarify` to your path)</br>
`cd C:\path\to\avatarify`</br></br>
* Mac/Linux (change `/path/to/avatarify` to your path)</br>
`cd /path/to/avatarify`
3. Copy-paste to the terminal the command below and run;
4. It can take some time to connect (usually up to 10 seconds). If the preview window doesn't appear in a minute or two, look for the errors above in this notebook and report in the [issues](https://github.com/alievk/avatarify/issues) or [Slack](https://join.slack.com/t/avatarify/shared_invite/zt-dyoqy8tc-~4U2ObQ6WoxuwSaWKKVOgg).

In [149]:
print('Copy-paste to the terminal the command below and run (press Enter)\n')
print('Mac:')
print(f'./run_mac.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nWindows:')
print(f'run_windows.bat --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nLinux:')
print(f'./run.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')

Copy-paste to the terminal the command below and run (press Enter)

Mac:


NameError: name 'in_addr' is not defined

### Action Required: Update ngrok Authtoken

The `open_tunnel_ngrok.sh` script reported an `ERR_NGROK_107` error, indicating that the provided ngrok authtoken is invalid.

**Please go to cell `YDtPpi77AkQ1` and replace the placeholder authtoken with your valid ngrok authtoken from your ngrok dashboard ([https://dashboard.ngrok.com/get-started/your-authtoken](https://dashboard.ngrok.com/get-started/your-authtoken)).**

Once updated, please run the following cells to re-initialize the ngrok configuration and attempt to open the tunnel again.

In [147]:
import os

config =\
f"""
version: 2
authtoken: {authtoken}
region: {region}
console_ui: False
tunnels:
  input:
    addr: {local_in_port}
    proto: tcp
  output:
    addr: {local_out_port}
    proto: tcp
"""

# Ensure ngrok.conf is written to the expected location by ngrok
config_path = os.path.join('/content/avatarify/avatarify', 'ngrok.conf')
with open(config_path, 'w') as f:
  f.write(config)

In [145]:
import os
import time
from subprocess import Popen, PIPE, TimeoutExpired

# Ensure we are in the correct directory for ngrok scripts
os.chdir('/content/avatarify/avatarify')

print(f"Current working directory: {os.getcwd()}")

# Construct absolute path to the script
script_path = os.path.join(os.getcwd(), 'scripts', 'open_tunnel_ngrok.sh')
print(f"Attempting to run script from: {script_path}")

# (Re)Open tunnel
proc = Popen(f'/bin/bash "{script_path}"', shell=True, stdout=PIPE, stderr=PIPE)
try:
  # Use communicate to wait for the bash script to exit, and capture its output
  stdout_val, stderr_val = proc.communicate(timeout=60)
  print(f"\n--- open_tunnel_ngrok.sh stdout ---\n{stdout_val.decode()}")
  print(f"\n--- open_tunnel_ngrok.sh stderr ---\n{stderr_val.decode()}")
  print(f"open_tunnel_ngrok.sh exit code: {proc.returncode}")
except TimeoutExpired:
  proc.kill()
  stdout_val, stderr_val = proc.communicate()
  print(f"\n--- open_tunnel_ngrok.sh stdout (Timeout) ---\n{stdout_val.decode()}")
  print(f"\n--- open_tunnel_ngrok.sh stderr (Timeout) ---\n{stderr_val.decode()}")
  print("Error: open_tunnel_ngrok.sh timed out.")


time.sleep(45) # Give ngrok time to fully initialize and write to its log

# Read ngrok output log
print("\n--- ngrok_output.log content ---\n")
if os.path.exists('ngrok_output.log'):
    with open('ngrok_output.log', 'r') as f:
        print(f.read())
else:
    print("ngrok_output.log not found.")

# Check if ngrok process is running
print("\nChecking for ngrok process:\n")
!ps aux | grep ngrok | grep -v grep

Current working directory: /content/avatarify/avatarify
Attempting to run script from: /content/avatarify/avatarify/scripts/open_tunnel_ngrok.sh

--- open_tunnel_ngrok.sh stdout ---
[DEBUG] Starting open_tunnel_ngrok.sh
[DEBUG] Attempting to kill previous ngrok processes...
[DEBUG] Finished killing ngrok processes.
[DEBUG] Current directory for ngrok start: /content/avatarify/avatarify
[DEBUG] Starting ngrok with config ngrok.conf
[DEBUG] Ngrok process started in background.
[DEBUG] open_tunnel_ngrok.sh finished.


--- open_tunnel_ngrok.sh stderr ---
+ echo '[DEBUG] Starting open_tunnel_ngrok.sh'
+ echo '[DEBUG] Attempting to kill previous ngrok processes...'
+ killall ngrok
ngrok: no process found
+ true
+ echo '[DEBUG] Finished killing ngrok processes.'
++ pwd
+ echo '[DEBUG] Current directory for ngrok start: /content/avatarify/avatarify'
+ echo '[DEBUG] Starting ngrok with config ngrok.conf'
+ echo '[DEBUG] Ngrok process started in background.'
+ echo '[DEBUG] open_tunnel_ngrok.sh 

In [146]:
# Get tunnel addresses
try:
  in_addr, out_addr = get_tunnel_adresses()
  print("Tunnel opened")
except Exception as e:
  # This block assumes ps is still defined from previous executions, or you'd need to re-run a ps command.
  # For robustness, we'll ensure any potential ps.stdout.readlines() is handled if ps isn't available.
  # However, in_addr/out_addr not being defined is the primary issue here.
  print("Something went wrong, reopen the tunnel or check ngrok output.")
  print(f"Error: {type(e).__name__}: {e}")

Attempting to get tunnel addresses...
Raw curl output: []
Something went wrong, reopen the tunnel or check ngrok output.
Error: AssertionError: Curl command returned no info.


# Logs

If something doesn't work as expected, please run the cells below and include the logs in your report.

In [43]:
#@title
!cat ./var/log/cam_fomm.log | head -100

[1767040717.808178] Loading Predictor


In [44]:
#@title
!cat ./var/log/recv_worker.log | tail -100

cat: ./var/log/recv_worker.log: No such file or directory


In [45]:
#@title
!cat ./var/log/predictor_worker.log | tail -100

cat: ./var/log/predictor_worker.log: No such file or directory


In [46]:
#@title
!cat ./var/log/send_worker.log | tail -100

cat: ./var/log/send_worker.log: No such file or directory


# Task
Add a new code cell to list the files in the `/content/avatarify` directory to verify the presence of `run.sh`, then re-run the cell that attempts to start the worker script.

## Verify cloned repository contents

### Subtask:
Add a new cell to list the files in the `/content/avatarify` directory to ensure `run.sh` is present and the repository was cloned correctly.


**Reasoning**:
To verify if the `run.sh` file is present and the repository was cloned correctly, I will list the contents of the `/content/avatarify` directory as requested.



In [47]:
!ls -F /content/avatarify

avatarify/


**Reasoning**:
The previous `!ls -F /content/avatarify` command only showed `avatarify/`, indicating that the actual cloned repository contents (including `run.sh`) are likely in a nested directory, `/content/avatarify/avatarify`. I need to list the files in this nested directory to confirm the presence of `run.sh` as per the task instructions.



In [48]:
!ls -F /content/avatarify/avatarify

afy/		 LICENSE.md			  run.sh*
avatarify.ipynb  ngrok*				  run_windows.bat
avatars/	 ngrok-v3-stable-linux-amd64.tgz  scripts/
config.yaml	 README.md			  var/
Dockerfile	 requirements_client.txt	  vox-adv-cpk.pth.tar
docs/		 requirements.txt
fomm/		 run_mac.sh*


**Reasoning**:
I have successfully verified that `run.sh` is present in `/content/avatarify/avatarify`. Now, I need to re-run the cell that attempts to start the worker script. The previous error indicated that `./run.sh` was not found in `/content/avatarify`. To fix this, I will update the `os.chdir` command to navigate into the correct nested `avatarify` directory, `/content/avatarify/avatarify`, before trying to execute `run.sh`.



In [38]:
import os
from subprocess import Popen, PIPE
import shlex
import time

# Ensure we are in the correct directory
os.chdir('/content/avatarify/avatarify')

# Ensure run.sh exists and is executable
if not os.path.exists('./run.sh'):
    print("Error: run.sh not found. Please ensure Avatarify was cloned correctly.")
else:
    os.chmod('./run.sh', 0o755) # Make it executable
    print("run.sh found and set to executable.")

# (Re)Start the worker
with open('/tmp/run.txt', 'w') as f:
  ps = Popen(
      shlex.split(f'./run.sh --is-worker --in-port {local_in_port} --out-port {local_out_port} --no-vcam --no-conda'),
      stdout=f, stderr=f)
  time.sleep(3)


run.sh found and set to executable.


**Reasoning**:
The previous step successfully re-ran the worker script after fixing the directory issue. Now, I need to verify that the worker has successfully started by checking the running processes, as per the original notebook's sequence of steps.



In [39]:
!ps aux | grep 'python3 afy/cam_fomm.py' | grep -v grep | tee /tmp/ps_run
!if [[ $(cat /tmp/ps_run | wc -l) == "0" ]]; then echo "Worker failed to start"; cat /tmp/run.txt; else echo "Worker started"; fi

root        2373 22.7  3.5 4246320 475280 ?      Rl   20:38   0:03 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
Worker started


## Summary:

### Q&A
The task was to verify the presence of `run.sh` in the `/content/avatarify` directory and then re-run the worker script. This was successfully achieved.

### Data Analysis Key Findings
*   Initial inspection of `/content/avatarify` revealed that the repository contents, including `run.sh`, were nested within a subfolder named `avatarify` (i.e., `/content/avatarify/avatarify/run.sh`).
*   The `run.sh` script was confirmed to be present at `/content/avatarify/avatarify/run.sh`.
*   The `run.sh` script was successfully made executable.
*   The Avatarify worker process, identified as `python3 afy/cam_fomm.py`, was successfully started and confirmed to be running.

### Insights or Next Steps
*   When cloning repositories, it's crucial to verify the exact directory structure, especially if a repository name matches a top-level folder, as files might be nested one level deeper than expected.
*   The worker script successfully initiated, indicating that the core component for the avatarify service is operational.
